# Matrix Conjectures with TxGraffiti

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/RandyRDavila/AI-discovery-in-mathematics-with-TxGraffiti/blob/main/notebooks/matrices.ipynb)

## Introduction

This notebook applies the TxGraffiti algorithm to generate conjectures on square symmetric matrices. Matrices are fundamental in various areas of mathematics and applied sciences, including linear algebra, statistics, and machine learning. By examining the relationships between different matrix properties, TxGraffiti aims to uncover new mathematical insights and conjectures.

## Dataset

The dataset consists of square symmetric matrices with various numerical properties such as:
- **Numerical Properties**: Trace, determinant, second largest eigenvalue, Frobenius norm, condition number, and condition number times Frobenius norm.
- **Boolean Properties**: Is positive definite, is sparse.

## Objectives

- Generate conjectures relating different numerical properties of square symmetric matrices.
- Identify significant relationships and patterns in matrix properties.
- Apply the Theo and Static Dalmatian heuristics to filter and refine the conjectures.

## Usage

1. **Run the cells to generate the dataset and apply TxGraffiti.**
2. **Examine the generated conjectures and their significance.**

Explore the fascinating properties of square symmetric matrices and discover new mathematical conjectures with TxGraffiti.

---

In [6]:
# If running in Google Colab, you will need to pip install pulp.
# !pip install pulp

# Import the necessary libraries.
import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer
from pulp import *
from fractions import Fraction
from itertools import combinations

# Define the hypothesis, conclusion, and conjecture classes
class Hypothesis:
    def __init__(self, statements):
        self.statements = statements

class LinearConclusion:
    def __init__(self, target, inequality, slope, other, intercept):
        self.target = target
        self.inequality = inequality
        self.slope = slope
        self.other = other
        self.intercept = intercept

class LinearConjecture:
    def __init__(self, hypothesis, conclusion, symbol, touch, type="symmetric matrix"):
        self.hypothesis = hypothesis
        self.conclusion = conclusion
        self.symbol = symbol
        self.touch = touch
        self.type = type

    def __repr__(self):
        if self.hypothesis.statements:
            hypothesis_str = " and ".join([f"{self.symbol} is {h}" for h in self.hypothesis.statements])
            return (f"For any {self.type} {self.symbol}, if {hypothesis_str}, then "
                    f"{self.conclusion.target}({self.symbol}) {self.conclusion.inequality} "
                    f"{self.conclusion.slope}*{self.conclusion.other}({self.symbol}) + "
                    f"{self.conclusion.intercept}, with equality on {self.touch} instances.")
        else:
            return (f"For any {self.type} {self.symbol}, "
                    f"{self.conclusion.target}({self.symbol}) {self.conclusion.inequality} "
                    f"{self.conclusion.slope}*{self.conclusion.other}({self.symbol}) + "
                    f"{self.conclusion.intercept}, with equality on {self.touch} instances.")

    def get_sharp_objects(self, df):
        X = df[self.conclusion.other].to_numpy()
        Y = df[self.conclusion.target].to_numpy()
        sharp_indices = df[np.isclose(Y, float(self.conclusion.slope) * X + float(self.conclusion.intercept))].index
        return df.loc[sharp_indices]

    def calculate_distances(self, df):
        X = df[self.conclusion.other].to_numpy()
        Y = df[self.conclusion.target].to_numpy()
        distances = np.abs(Y - (float(self.conclusion.slope) * X + float(self.conclusion.intercept)))
        return distances

def make_upper_linear_conjecture(df, target, other, hypothesis, symbol="A"):
    for hyp in hypothesis:
        df = df[df[hyp] == True]
    X = df[other].to_numpy()
    Y = df[target].to_numpy()

    prob = LpProblem("UpperBoundConjecture", LpMinimize)
    w = LpVariable("w")
    b = LpVariable("b")

    prob += lpSum([w * x + b - y for x, y in zip(X, Y)])

    for x, y in zip(X, Y):
        prob += w * x + b - y >= 0

    prob.solve()

    if w.varValue is None or b.varValue is None:
        return None

    m = Fraction(w.varValue).limit_denominator(10)
    b = Fraction(b.varValue).limit_denominator(10)
    if m == 0:
        return None  # Skip trivial conjectures

    touch = np.sum(np.isclose(Y, float(m) * X + float(b)))

    hypothesis = Hypothesis(hypothesis)
    conclusion = LinearConclusion(target, "<=", m, other, b)

    return LinearConjecture(hypothesis, conclusion, symbol, touch)

def make_lower_linear_conjecture(df, target, other, hypothesis, symbol="A"):
    for hyp in hypothesis:
        df = df[df[hyp] == True]
    X = df[other].to_numpy()
    Y = df[target].to_numpy()

    prob = LpProblem("LowerBoundConjecture", LpMaximize)
    w = LpVariable("w")
    b = LpVariable("b")

    prob += lpSum([w * x + b - y for x, y in zip(X, Y)])

    for x, y in zip(X, Y):
        prob += w * x + b - y <= 0

    prob.solve()

    if w.varValue is None or b.varValue is None:
        return None

    m = Fraction(w.varValue).limit_denominator(10)
    b = Fraction(b.varValue).limit_denominator(10)
    if m == 0:
        return None  # Skip trivial conjectures

    touch = np.sum(np.isclose(Y, float(m) * X + float(b)))

    hypothesis = Hypothesis(hypothesis)
    conclusion = LinearConclusion(target, ">=", m, other, b)

    return LinearConjecture(hypothesis, conclusion, symbol, touch)

def make_all_upper_linear_conjectures(df, target, others, properties):
    conjectures = []
    for other in others:
        for k in range(4):  # Considering hypotheses of none, one, two, and three boolean properties
            for prop_comb in combinations(properties, k):
                if other != target:
                    conjecture = make_upper_linear_conjecture(df, target, other, prop_comb)
                    if conjecture:
                        conjectures.append(conjecture)
    return conjectures

def make_all_lower_linear_conjectures(df, target, others, properties):
    conjectures = []
    for other in others:
        for k in range(4):  # Considering hypotheses of none, one, two, and three boolean properties
            for prop_comb in combinations(properties, k):
                if other != target:
                    conjecture = make_lower_linear_conjecture(df, target, other, prop_comb)
                    if conjecture:
                        conjectures.append(conjecture)
    return conjectures

def sort_by_touch_number(conjectures):
    return sorted(conjectures, key=lambda x: x.touch, reverse=True)

def apply_theo_heuristic(conjectures):
    filtered_conjectures = []
    for conj_1 in conjectures:
        is_general = True
        for conj_2 in filtered_conjectures:
            if (conj_1.conclusion.slope == conj_2.conclusion.slope and
                conj_1.conclusion.intercept == conj_2.conclusion.intercept and
                conj_1.conclusion.inequality == conj_2.conclusion.inequality and
                set(conj_1.hypothesis.statements).issubset(set(conj_2.hypothesis.statements))):
                is_general = False
                break
        if is_general:
            filtered_conjectures.append(conj_1)
    return filtered_conjectures

def apply_static_dalmatian_heuristic(df, conjectures):
    filtered_conjectures = []
    for conj in conjectures:
        conj_distances = conj.calculate_distances(df)
        keep_conj = True
        for other_conj in filtered_conjectures:
            other_distances = other_conj.calculate_distances(df)
            if np.all(conj_distances >= other_distances):
                keep_conj = False
                break
        if keep_conj:
            filtered_conjectures.append(conj)
    return filtered_conjectures

def txgraffiti_conjecture_generation(df, targets, invariants, properties):
    conjectures = []
    for target in targets:
        upper_conjectures = make_all_upper_linear_conjectures(df, target, invariants, properties)
        lower_conjectures = make_all_lower_linear_conjectures(df, target, invariants, properties)
        conjectures += upper_conjectures + lower_conjectures

    conjectures = sort_by_touch_number(conjectures)
    conjectures = apply_theo_heuristic(conjectures)
    conjectures = apply_static_dalmatian_heuristic(df, conjectures)

    return conjectures

# Function to generate a square symmetric matrix
def generate_square_symmetric_matrix(size):
    A = np.random.rand(size, size)
    return (A + A.T) / 2

# Generate data for square symmetric matrices
n_samples = 1000
matrix_size = 20

data = []
for _ in range(n_samples):
    matrix = generate_square_symmetric_matrix(matrix_size)
    trace = np.trace(matrix)
    determinant = np.linalg.det(matrix)
    eigvals = np.linalg.eigvals(matrix)
    second_largest_eigenvalue = np.partition(eigvals, -2)[-2]
    frobenius_norm = np.linalg.norm(matrix, 'fro')
    condition_number = np.linalg.cond(matrix)
    condition_number_times_frobenius_norm = condition_number * frobenius_norm
    is_positive_definite = np.all(eigvals > 0)
    is_sparse = (np.count_nonzero(matrix) / (matrix_size**2)) < 0.5
    data.append([trace, determinant, second_largest_eigenvalue, frobenius_norm, condition_number, is_positive_definite, is_sparse, condition_number_times_frobenius_norm])

columns = ["trace", "determinant", "second_largest_eigenvalue", "frobenius_norm", "condition_number", "is_positive_definite", "is_sparse", "condition_number_times_frobenius_norm"]
df = pd.DataFrame(data, columns=columns)

# Define the targets, invariants, and properties
targets = ["trace", "determinant", "second_largest_eigenvalue", "frobenius_norm", "condition_number"]
invariants = ["trace", "determinant", "second_largest_eigenvalue", "frobenius_norm", "condition_number", "condition_number_times_frobenius_norm"]
properties = ["is_positive_definite", "is_sparse"]

# Generate conjectures using the TxGraffiti algorithm
conjectures = txgraffiti_conjecture_generation(df, targets, invariants, properties)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/randydavila/Documents/Automated-Conjecturing/AI-discovery-in-mathematics-with-TxGraffiti/env/lib/python3.11/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/92/bxgdy2896wdgw0bx9f_1ghhh0000gn/T/6e1fc22200fa4680a7a76f2cf11248c7-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/92/bxgdy2896wdgw0bx9f_1ghhh0000gn/T/6e1fc22200fa4680a7a76f2cf11248c7-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 1005 COLUMNS
At line 3008 RHS
At line 4009 BOUNDS
At line 4012 ENDATA
Problem MODEL has 1000 rows, 2 columns and 2000 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 1000 (0) rows, 2 (0) columns and 2000 (0) elements
0  Obj 0 Primal inf 1496274.1 (1000) Dual inf 0.16155665 (2) w.o. free dual inf (0)
2  Obj 14098.704
2  Obj -3.350753e+11 Primal inf 5.2718281e+13 (548) Dual inf 3.4812

In [7]:
# Print the generated conjectures
for i, conj in enumerate(conjectures[:20]):
    print(f"Conjecture {i+1}. ", conj, "\n")

Conjecture 1.  For any symmetric matrix A, condition_number(A) <= 140918*second_largest_eigenvalue(A) + -986176/7, with equality on 2 instances. 

Conjecture 2.  For any symmetric matrix A, condition_number(A) <= 175778/7*trace(A) + -564909/4, with equality on 1 instances. 

Conjecture 3.  For any symmetric matrix A, condition_number(A) <= 14463387/2*determinant(A) + 463827/4, with equality on 1 instances. 

Conjecture 4.  For any symmetric matrix A, condition_number(A) <= -942553/7*frobenius_norm(A) + 7825624/5, with equality on 1 instances. 

Conjecture 5.  For any symmetric matrix A, trace(A) <= 1679/10*determinant(A) + 141/10, with equality on 0 instances. 

Conjecture 6.  For any symmetric matrix A, trace(A) <= 20/3*second_largest_eigenvalue(A) + 31/10, with equality on 0 instances. 

Conjecture 7.  For any symmetric matrix A, trace(A) <= 2*frobenius_norm(A) + -8, with equality on 0 instances. 

Conjecture 8.  For any symmetric matrix A, trace(A) >= -1397/4*determinant(A) + 45/8, 